# Orlando - Classificação Binária - Tuning dos parâmetros

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, InputLayer
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import BinaryCrossentropy, Hinge
from tensorflow.keras import backend as K
import tensorflow as tf

# Ativa o modo de execução ansiosa (necessário para usar .numpy() em ambientes com erro NotImplementedError)
tf.config.run_functions_eagerly(True)


# 1. Carregamento dos dados
X = pd.read_csv(r'C:\Users\CLIENTE\OneDrive\PROJETOS\PROJETO_DEEP_LEARNING\Parte 1 - Redes Neurais Artificiais\classificação binária\entradas_breast.csv')
y = pd.read_csv(r'C:\Users\CLIENTE\OneDrive\PROJETOS\PROJETO_DEEP_LEARNING\Parte 1 - Redes Neurais Artificiais\classificação binária\saidas_breast.csv')

# 2. Função para construir a rede com hiperparâmetros variáveis
def criar_rede(optimizer, loss, kernel_initializer, activation, neurons):
    K.clear_session()
    model = Sequential()
    model.add(InputLayer(input_shape=(30,)))
    model.add(Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer))
    model.add(Dropout(0.2))
    model.add(Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='sigmoid'))
    # Cria uma nova instância do otimizador a cada chamada
    opt = Adam() if optimizer == 'adam' else SGD()
    model.compile(optimizer=opt, loss=loss, metrics=['binary_accuracy'])
    return model


# 3. Parâmetros a serem testados
parametros = {
    'optimizer': ['adam', 'sgd'],  # agora como strings
    'loss': ['binary_crossentropy', 'hinge'],
    'kernel_initializer': ['random_uniform', 'normal'],
    'activation': ['relu', 'tanh'],
    'neurons': [8, 16],
    'batch_size': [10, 30],
    'epochs': [50, 100]
}

# 4. Validação cruzada com 3 folds (execução reduzida para performance)
resultados = []
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# 5. Loop manual simulando GridSearchCV
for optimizer in parametros['optimizer']:
    for loss in parametros['loss']:
        for init in parametros['kernel_initializer']:
            for activation in parametros['activation']:
                for neurons in parametros['neurons']:
                    for batch_size in parametros['batch_size']:
                        for epochs in parametros['epochs']:
                            acuracias = []
                            for train_idx, test_idx in kfold.split(X, y):
                                X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
                                y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

                                model = criar_rede(optimizer, loss, init, activation, neurons)
                                model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0)

                                preds = model.predict(X_test)
                                preds = (preds > 0.5)
                                acc = accuracy_score(y_test, preds)
                                acuracias.append(acc)

                            media = np.mean(acuracias)
                            resultados.append({
                                'optimizer': optimizer,
                                'loss': loss,
                                'kernel_initializer': init,
                                'activation': activation,
                                'neurons': neurons,
                                'batch_size': batch_size,
                                'epochs': epochs,
                                'mean_accuracy': media
                            })

# 6. Identificar os melhores parâmetros
df_resultados = pd.DataFrame(resultados)
melhor = df_resultados.loc[df_resultados['mean_accuracy'].idxmax()]
df_resultados_sorted = df_resultados.sort_values(by='mean_accuracy', ascending=False).head(5)

print("Top 5 Combinações de Hiperparâmetros:")
print(df_resultados_sorted)
melhor


c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow

...

c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\CLIENTE\anaconda3\envs\orlando\lib\site-packages\tensorflow